In [3]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv("../files/hr_raw_data_v2.csv") #NO hay que poner index col = 0 porque si no, nos devuelve el "employee_number" como índice

In [5]:
df.head() #ahora sí está bien, y el shape ahora será 33 columnas en vez de 32 como daba antes

,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,departured,year_at_comp,...,monthly_income,daily_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,M,1972,51.0,Unknown,6,Research Director,Research & Development,No,20,...,16280.830000,2015.722222,13,1,3,3,3,3.0,7,NaN
1,2,M,1971,52.0,Unknown,1,Manager,Unknown,No,33,...,16125.406667,2063.388889,14,3,2,3,1,3.0,0,34.0
2,3,M,1981,42.0,Married,4,Manager,Research & Development,No,22,...,15943.720000,1984.253968,11,3,3,4,4,NaN,1,22.0
3,4,F,1976,47.0,Married,2,Research Director,Research & Development,No,20,...,14307.500000,1771.404762,19,1,3,3,2,NaN,3,NaN
4,5,F,1977,46.0,Divorced,3,Sales Executive,Sales,No,19,...,12783.920000,1582.771346,12,1,4,1,4,3.0,2,NaN


In [6]:
df.shape #32 columnas pero en la query tenemos 33

(1614, 33)

In [7]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
host='127.0.0.1',
auth_plugin ='mysql_native_password')
mycursor = cnx.cursor()

In [8]:
mycursor.execute("CREATE DATABASE ABC_Corporation")

In [9]:
mycursor.execute("USE ABC_Corporation")

In [10]:
cnx.close()

In [11]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', 
                                host='127.0.0.1', 
                                database='abc_corporation', 
                                auth_plugin ='mysql_native_password')

In [12]:
mycursor = cnx.cursor()

In [13]:
mycursor.execute("""CREATE TABLE EmployeeInfo(
  employee_number INT NOT NULL,
  gender ENUM('M', 'F') NULL,
  birth_year INT NULL,
  age FLOAT NULL,
  marital_status ENUM('Single', 'Divorced', 'Married', 'Unknown') NULL,
  dist_home INT NULL,
  job_title VARCHAR(50) NULL,
  department VARCHAR(50) NULL,
  departured ENUM('Yes', 'No') NULL,
  year_at_comp INT NULL,
  standard_hours VARCHAR(45) NULL,
  remote ENUM('Yes', 'No') NULL,
  business_travel ENUM('Unknown', 'travel_rarely', 'travel_frequently', 'non-travel') NULL,
  over_time ENUM('Yes', 'No', 'Unknown') NULL,
  job_level INT NULL,
  stock_opt_level INT NULL,
  traning_times_last_year INT NULL,
  perf_rate FLOAT NULL,
  year_last_promotion INT NULL,
  year_current_mngr INT NULL,
  education_field VARCHAR(45) NULL,
  education_scale INT NULL,
  annual_salary FLOAT NOT NULL,
  monthly_income FLOAT NULL,
  daily_rate FLOAT NULL,
  perc_salary_hike INT NULL,
  env_sat_rate INT NULL,
  job_involvement INT NULL,
  job_sat_rate INT NULL,
  relationship_sat_rate INT NULL,
  work_life_balance FLOAT NULL,
  num_comp_worked INT NULL,
  tot_working_year FLOAT NULL,
PRIMARY KEY (employee_number))""");

In [14]:
cnx.close()

In [ ]:
cnx = mysql.connector.connect(user='root', password='', 
                                host='127.0.0.1', 
                                database='abc_corporation', 
                                auth_plugin ='mysql_native_password')

In [16]:
mycursor = cnx.cursor()

In [17]:
#OJO! las columnas aquí tienen que ir sin comillas simples, lo modifico.
#OJO otra comilla al principio del %: VALUES ('%s, ==> lo quito
query = """INSERT INTO EmployeeInfo (employee_number, gender, birth_year, age, marital_status,
    dist_home, job_title, department, departured, year_at_comp,
    standard_hours, remote, business_travel, over_time, job_level,
    stock_opt_level, traning_times_last_year, perf_rate,
    year_last_promotion, year_current_mngr, education_field,
    education_scale, annual_salary, monthly_income, daily_rate,
    perc_salary_hike, env_sat_rate, job_involvement, job_sat_rate,
    relationship_sat_rate, work_life_balance, num_comp_worked,
    tot_working_year) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""" 


In [18]:
lista_de_tuplas = [tuple(row) for row in df.to_records(index=False)]

In [19]:
#OJO, las tuplas en python son inmutables, por eso vamos a hacer una lista temporal donde guardaremos las nuevas tuplas:

nueva_lista_de_tuplas = []

for element in lista_de_tuplas:

    lista_temporal = [] # Lista temporal para construir la nueva tupla

    for i in element:
        if isinstance(i, str):  
            lista_temporal.append(i)  # Si es str, lo dejamos igual
        
        elif isinstance(i, float) and np.isnan(i):  
            lista_temporal.append(0)  # Si es un NaN, lo cambiamos por 0, porque si no, sigue siendo "np.nan" y nos salta en SQL

        else:
            lista_temporal.append(int(i))  # Convertimos a int si es posible (salta Error en los NaN)
    
    nueva_lista_de_tuplas.append(tuple(lista_temporal))  # Convertimos la lista_temporal en tupla y la metemos a la nueva lista


print(nueva_lista_de_tuplas[0])

(1, 'M', 1972, 51, 'Unknown', 6, 'Research Director', 'Research & Development', 'No', 20, 'Full Time', 'Yes', 'Unknown', 'No', 5, 0, 5, 3, 15, 15, 'Unknown', 3, 195370, 16280, 2015, 13, 1, 3, 3, 3, 3, 7, 0)


In [20]:
print("filas:", len(nueva_lista_de_tuplas))
print("col:", len(nueva_lista_de_tuplas[0]))

filas: 1614
col: 33


In [21]:
type(nueva_lista_de_tuplas[0][6])

str

In [22]:
mycursor.executemany(query, nueva_lista_de_tuplas) 
cnx.commit()
print("Datos insertados correctamente ✅.")
print(mycursor.rowcount, "registros insertados")

Datos insertados correctamente ✅.
1614 registros insertados


In [23]:
cnx.close()

In [24]:
df.head(10)

,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,departured,year_at_comp,...,monthly_income,daily_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,M,1972,51.0,Unknown,6,Research Director,Research & Development,No,20,...,16280.830000,2015.722222,13,1,3,3,3,3.0,7,NaN
1,2,M,1971,52.0,Unknown,1,Manager,Unknown,No,33,...,16125.406667,2063.388889,14,3,2,3,1,3.0,0,34.0
2,3,M,1981,42.0,Married,4,Manager,Research & Development,No,22,...,15943.720000,1984.253968,11,3,3,4,4,NaN,1,22.0
3,4,F,1976,47.0,Married,2,Research Director,Research & Development,No,20,...,14307.500000,1771.404762,19,1,3,3,2,NaN,3,NaN
4,5,F,1977,46.0,Divorced,3,Sales Executive,Sales,No,19,...,12783.920000,1582.771346,12,1,4,1,4,3.0,2,NaN
5,6,F,1975,48.0,Unknown,22,Manager,Research & Development,No,22,...,14311.670000,1771.920635,11,4,3,4,2,3.0,3,NaN
6,7,F,1964,59.0,Unknown,25,Sales Executive,Sales,No,21,...,8339.320000,1032.487286,11,1,3,1,4,2.0,7,28.0
7,8,M,1981,42.0,Married,1,Sales Executive,Sales,No,20,...,4492.840000,556.256661,25,2,3,3,3,3.0,1,20.0
8,9,F,1982,41.0,Married,2,Manager,Unknown,No,18,...,13829.170000,1712.182540,16,2,3,1,2,3.0,7,22.0
9,10,M,1982,41.0,Unknown,9,Research Director,Research & Development,No,18,...,15943.720000,1973.984127,17,1,3,3,2,4.0,2,21.0
